# 다이소 재고 검색 프로젝트

- 기능 1. 해당 상품명 검색 시 추천순(or 판매량, 리뷰많은순) 상품 5개 선택
- 기능 2. 선택된 상품들에 대해서 리뷰 데이터 수집 (별점, 내용)
- 기능 3. 입력받은 지역 근처 다이소 매장 검색 (3개)
- 기능 4. 선택된 상품과 선택한 매장에 대해 재고 검색

> 입력: 상품명, 지역  
> 출력: 상품 리스트와 리뷰, 재고

다이소몰 URL: https://www.daisomall.co.kr/ds

### 입력 받기 & 필요한 모듈 임포트

In [1]:
import time
import bs4
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd
from IPython.display import clear_output 

### 상품 입력해서 상위 5개 항목의 정보 가져오는 함수


In [2]:
def get_products_info(good, browser) :
    browser.get('https://www.daisomall.co.kr/ds/dst/SCR_DST_0015?searchTerm='+good)
    time.sleep(2)
    goods = browser.find_elements(By.CLASS_NAME, "goods-unit")
    
    names = []
    prices = []
    links = []
    
    for good in goods[0:5] :
        names.append(good.find_element(By.CLASS_NAME, "tit").text.replace("BEST","").strip())
        prices.append(good.find_element(By.CLASS_NAME, "value").text)
        links.append(good.find_element(By.TAG_NAME, "a").get_attribute("href"))
        
    data = {"names" : names, "prices" : prices, "links" : links}
    df = pd.DataFrame(data=data)
    
    return df

### 상품의 상세 리뷰 가져오는 함수

In [3]:
def get_review(url, browser) :
    browser.get(url)
    time.sleep(2)
    
    response = browser.page_source
    
    review_page = bs4.BeautifulSoup(response)
    
    one_page_all_reviews = review_page.find_all('div',{'class':'cont'})
    one_page_all_scores = review_page.find_all('span',{'class':'score'})
    
    if not one_page_all_reviews :
        return pd.DataFrame()
    
    else:
        score_list = [score.get_text(strip = True) for score in one_page_all_scores]
        review_list = [review.get_text(strip = True) for review in one_page_all_reviews]
    
    df = pd.DataFrame({'text': review_list, 'score': score_list})
    
    return df


### 해당 지역의 다이소 매장 찾는 함수

In [4]:
def get_store(region, browser):
    search_url = 'https://www.daisomall.co.kr/ms/msg/SCR_MSG_0019'
    browser.get(search_url)

    time.sleep(3) 
    
    # 광고 제거
    btn = browser.find_element(by=By.XPATH, value='//*[@id="btn-close-nday"]')
    btn.click()
    time.sleep(1)
    
    # 매장 검색 박스
    search_box = browser.find_elements(by=By.CLASS_NAME, value='el-input__inner')[1] # index 0: product search box
    search_box.clear()
    search_box.send_keys(region)
    search_box.send_keys(Keys.ENTER)
    time.sleep(3)
    
    # 매장 당 정보 가져오기
    region_boxies = browser.find_elements(by=By.CLASS_NAME, value='store-map')
    store_list = []
    
    for i, region_box in enumerate(region_boxies):
        store = region_box.find_element(by=By.CLASS_NAME, value='tit-h5').text
        location = region_box.find_element(by=By.XPATH, value='//*[@id="pane-2"]/div/div[2]/div[{}]/div[2]/div[2]'.format(i+1)).text
        info = region_box.find_element(by=By.CLASS_NAME, value='info-group')
        opening = info.find_elements(by=By.TAG_NAME, value='span')[0].text.replace('영업 시간 ', '')
        tel = info.find_elements(by=By.TAG_NAME, value='span')[1].text.replace('매장 전화 ', '')
    
        store_list.append({'store':store.split()[0], 'location':location, 'opening':opening, 'tel':tel})
        
        store_df = pd.DataFrame(store_list)
        
    return store_df

### 특정 상품의 특정 매장에서의 재고 검색

In [13]:
def get_stock(product_no, store, browser):
    url = 'https://www.daisomall.co.kr/ms/msg/SCR_MSG_0019'
    browser.get(url)
    time.sleep(3)
    
    # step 1. 매장 상품 찾기 클릭
    btn = browser.find_element(by=By.CSS_SELECTOR, value='#tab-3 > button')
    btn.click()
    time.sleep(2)
    
    # step 2. 특정 상품 검색 및 선택
    search_box = browser.find_element(by=By.CSS_SELECTOR, value='#pane-3 > div > div.form-wrap > div:nth-child(1) > form > div > div > div > div > input')
    search_box.clear()
    search_box.send_keys(product_no)
    search_box.send_keys(Keys.ENTER)
    time.sleep(2)
    
    check_btn = browser.find_element(by=By.CLASS_NAME, value='goods-check')
    check_btn = check_btn.find_element(by=By.TAG_NAME, value='label')
    check_btn.click()
    time.sleep(2)
    
    choice_btn = browser.find_element(by=By.CSS_SELECTOR, value='#__layout > section > div.wrap.scrollArea > div > div:nth-child(3) > div > div.el-dialog__footer > div > button')
    choice_btn.click()
    time.sleep(2)
        
    # step 3. 해당 매장 검색 및 선택
    search_box = browser.find_element(by=By.CSS_SELECTOR, value="#pane-3 > div > div.form-wrap > div:nth-child(2) > form > div > div > div > div > div.el-input.el-input--suffix > input")
    search_box.clear()
    search_box.send_keys(store)
    search_box.send_keys(Keys.ENTER)
    time.sleep(2)
        
    check_btn = browser.find_element(by=By.CLASS_NAME, value='el-radio__input')
    check_btn.click()
    time.sleep(2)
    
    choice_btn = browser.find_element(by=By.CSS_SELECTOR, value='#__layout > section > div.wrap.scrollArea > div > div:nth-child(4) > div > div.el-dialog__footer > div > button')
    choice_btn.click()
    time.sleep(2)
    
    # step 4. 재고 정보 가져오기
    try:                    # 일반적인 유형의 결과 (재고 현황 or 품절 or 단종 or -)
        stock_info = browser.find_element(by=By.CSS_SELECTOR, value='#pane-3 > div > div.form-wrap > div.store-result-area > div > div.store-info > div.info-group').text
    except Exception as e:  # 취급하지 않는 상품의 결과
        stock_info = 'ZONE - 취급하지 않는 상품'
    
    # step 5. 재고 결과 처리
    stock_info = stock_info.split()
    zone = ' '.join(stock_info[:2])
    stock =  ' '.join(stock_info[2:])
    
    if '품절' in stock:
        stock = '품절'
    elif '단종' in stock:
        stock = '단종'
    elif stock == '-':
        stock = '-'
    elif stock == '취급하지 않는 상품':
        pass
    else:
        stock = ' '.join(stock.split()[1:3])
        
        
    return stock, zone

---

### 메인

In [7]:
# 브라우저 옵션 설정 및 브라우저 실행
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')                    
options.add_argument('--disable-dev-shm-usage')    

driver = webdriver.Edge(options = options)
driver.maximize_window()

# DataFrame 설정
pd.set_option('display.max_colwidth',500)

In [8]:
# 상품 정보
product = input('상품명을 입력해주세요\n')
df = get_products_info(product, driver)

# 리뷰 정보
review_ls = []
for url in df["links"]:
    review_ls.append(get_review(url,driver))

display(df)

,names,prices,links
0,블루나 청정제주 도톰한 물티슈 엠보싱 캡형 90매 두꺼운 물티슈,"2,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1027023
1,모나리자 녹차가 좋은 물티슈 30매,"1,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1025456
2,참 깨끗한 물티슈 120매,"1,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1000865
3,맘애쏙 바디샤워 물티슈 3매입,"2,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1040069
4,청소용 물티슈 20X30cm 30매,"1,000",https://www.daisomall.co.kr/pd/pdr/SCR_PDR_0001?pdNo=1021725


In [10]:
# 리뷰 출력
while True :
    answer = input("리뷰를 보고 싶은 상품을 숫자로 선택해주세요(0~4) 보고 싶은 상품이 없을 경우 \"N\"을 입력해주세요")
    clear_output(wait=True)
    if answer == "N" :
        break
    else :
        if review_ls[int(answer)].empty:
            print("리뷰가 없습니다.")
        else :
            display(review_ls[int(answer)])

,text,score
0,세정제 뿌리고 번거로울때 간편히 쓰기 좋아요,5점
1,매일청소하면서 사용하기 간편해요,5점
2,다른데는 사실모르겠는데 싱크데수전닦았는데광이나네요,5점
3,잘닦여서 좋아요!!,5점
4,찌든때 기름때 청소에 이거없으면 안되요,5점
5,넘촉촉하다보다는축축느낌??암튼 뭐 그런정도네요.,5점
6,기름때가 잘 지워져요.,5점
7,찌든때도 잘지워지고 좋아요,5점
8,청소가 수월해졌어요기름때 찌든때 싹~~~,5점
9,청소하려고 구입했어요,5점


In [11]:
# 매장 정보
region = input("매장을 찾고 싶은 곳의 지역을 입력해주세요.\n")

store = get_store(region, driver)
display(store)

,store,location,opening,tel
0,명동본점,서울특별시 중구 명동길 43 (명동1가),10:00 ~ 22:00,1522-4400
1,명동역점,서울특별시 중구 퇴계로 134-1(남산동3가),10:00 ~ 22:00,1522-4400
2,춘천명동점,강원도 춘천시 명동길 3-1 (조양동),10:00 ~ 22:00,1522-4400
3,부산화명점,부산광역시 북구 금곡대로 358 (화명동),10:00 ~ 22:00,1522-4400
4,청주봉명점,충청북도 청주시 흥덕구 덕암로30번길 8(봉명동),10:00 ~ 22:00,043-263-7776
5,광명사거리점,경기도 광명시 광명로 907 (광명동),09:30 ~ 22:00,1522-4400
6,대구안지랑점,대구광역시 남구 두류공원로 45(대명동),10:00 ~ 22:00,053-657-6502
7,대전한밭대점,대전광역시 유성구 학하서로121번길 55-13 (덕명동),10:00 ~ 22:00,042-825-5990
8,앞산순환로점,대구광역시 남구 앞산순환로 535 (대명동),10:00 ~ 22:00,053-652-6016
9,영대병원역점,대구광역시 남구 대명로 297 (대명동) 1층,10:00 ~ 22:00,053-626-5665


In [14]:
# 재고 검색
product_num = int(input("아까 본 상품 중 맘에 드는 상품을 숫자로 골라주세요.(0~4)\n"))
input_store = input("해당 상품을 찾고 싶은 매장명을 입력해주세요.\n")
stock, location = get_stock(df["links"][product_num][-7:], input_store, driver)

product_name = df.iloc[product_num].names
product_prices = df.iloc[product_num].prices

print('\n\n')
print(f'>>> 다이소 {input_store}의 {product_name}({product_prices}원) 재고는 {stock}입니다. (위치: {location})')




>>> 다이소 명동역점의 청소용 물티슈 20X30cm 30매(1,000원) 재고는 9개 이하입니다. (위치: ZONE 46)


In [ ]:
# 드라이버 종료
driver.quit()